<a href="https://colab.research.google.com/github/soumendra/cnn-visualisation/blob/master/notebooks/visualising_and_interpreting_cnns_workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

![](assets/promiscuous_dlmodel.png)

1. Build a small model
2. Visualisation technique 1
3. Visualisation technique 2
4. Visualisation technique 3
5. Real-life Demo

# Build a small MNIST Model

* Step 1: Build model
* Step 2: Compile
* Step 3: Train
* Step 4: Test

## Data Prep

In [ ]:
import numpy as np
import keras

from keras.datasets import mnist
from keras import backend as K

num_classes = 10


# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## The MNIST Model

In [3]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D

batch_size = 128
epochs = 5

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax', name='preds'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 167s 3ms/step - loss: 0.2375 - acc: 0.9259 - val_loss: 0.0500 - val_acc: 0.9840
Epoch 2/5
60000/60000 [==============================] - 355s 6ms/step - loss: 0.0798 - acc: 0.9761 - val_loss: 0.0368 - val_acc: 0.9876
Epoch 3/5
60000/60000 [==============================] - 182s 3ms/step - loss: 0.0628 - acc: 0.9809 - val_loss: 0.0344 - val_acc: 0.9883
Epoch 4/5
60000/60000 [==============================] - 180s 3ms/step - loss: 0.0513 - acc: 0.9841 - val_loss: 0.0300 - val_acc: 0.9902
Epoch 5/5
60000/60000 [==============================] - 171s 3ms/step - loss: 0.0424 - acc: 0.9868 - val_loss: 0.0281 - val_acc: 0.9903
Test loss: 0.02809865023120874
Test accuracy: 0.9903
